In [5]:
from mpi4py import MPI
import math
import time

def f(x):
    return math.sqrt(1 - x**2)

def compute_pi_mpi(N):
    comm = MPI.COMM_WORLD
    size = comm.Get_size()
    rank = comm.Get_rank()
    
    dx = 1.0 / N
    local_n = N // size
    local_start = rank * local_n
    local_end = local_start + local_n if rank != size - 1 else N
    
    local_start_time = MPI.Wtime()
    
    local_sum = sum(f(i * dx) * dx for i in range(local_start, local_end))
    total_sum = comm.reduce(local_sum, op=MPI.SUM, root=0)
    
    local_end_time = MPI.Wtime()

    max_time = comm.reduce(local_end_time - local_start_time, op=MPI.MAX, root=0)
    
    if rank == 0:
        return 4 * total_sum, max_time

In [6]:
if __name__ == "__main__":
    N = 1000000
    result = compute_pi_mpi(N)
    if MPI.COMM_WORLD.Get_rank() == 0:
        pi_approx, max_execution_time = result
        print(f"Approximated pi = {pi_approx}")
        print(f"Max Execution Time: {max_execution_time} seconds")

Approximated pi = 3.1415946524138114
Max Execution Time: 0.27675900000031106 seconds
